# Vertex AI API Contract Layer

This notebook defines the **contract layer** for the Vertex AI sentiment analysis project.

It contains:
- **Dataclasses**: Type-safe configuration objects
- **Enums**: Fixed sets of valid options  
- **Protocol classes**: Abstract interfaces defining method signatures

The actual implementation lives in `vertex_ai.example.ipynb`.

## Configuration Contracts

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple, Any, Protocol
from enum import Enum

@dataclass
class VertexAIConfig:
    """Configuration for Vertex AI services."""
    project_id: str
    location: str
    bucket_name: str
    credentials_path: str = "vertex-ai-key.json"
    staging_bucket: Optional[str] = None
    
    def __post_init__(self):
        if self.staging_bucket is None:
            self.staging_bucket = f"gs://{self.bucket_name}/staging"

@dataclass
class TrainingJobConfig:
    """Configuration for a training job."""
    display_name: str
    model_name: str = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    learning_rate: float = 2e-5
    batch_size: int = 32
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    num_epochs: int = 4
    max_length: int = 128
    machine_type: str = "n1-standard-4"
    accelerator_type: str = "NVIDIA_TESLA_T4"
    accelerator_count: int = 1

@dataclass
class HyperparameterTuningConfig:
    """Configuration for hyperparameter tuning."""
    display_name: str
    max_trial_count: int = 10
    parallel_trial_count: int = 2
    metric_id: str = "f1_macro"
    metric_goal: str = "maximize"
    search_space: Dict[str, Dict[str, Any]] = field(default_factory=dict)
    
    def __post_init__(self):
        if not self.search_space:
            self.search_space = {
                "learning_rate": {"type": "double", "min": 5e-6, "max": 5e-5, "scale": "log"},
                "batch_size": {"type": "discrete", "values": [16, 32]},
                "weight_decay": {"type": "double", "min": 0.001, "max": 0.1, "scale": "linear"},
                "warmup_ratio": {"type": "double", "min": 0.0, "max": 0.3, "scale": "linear"}
            }

class ModelType(Enum):
    """Supported model types."""
    ROBERTA_TWITTER = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    BERT_BASE = "bert-base-uncased"
    DISTILBERT = "distilbert-base-uncased"

print("[SUCCESS] Configuration contracts defined")

## Service Interface (Protocol)

The `VertexAIService` protocol defines the abstract interface for Vertex AI operations.
This allows different implementations to satisfy the same contract.

In [ ]:
# Note: We import aiplatform types for type hints only
# The actual implementation is in example.ipynb
try:
    from google.cloud import aiplatform
    AIPLATFORM_AVAILABLE = True
except ImportError:
    AIPLATFORM_AVAILABLE = False
    print("[INFO] google-cloud-aiplatform not installed, using placeholder types")

class VertexAIService(Protocol):
    """Abstract interface for Vertex AI operations.
    
    This protocol defines WHAT methods exist but not HOW they work.
    The actual implementation is provided in vertex_ai.example.ipynb.
    """
    
    def initialize(self, config: VertexAIConfig) -> None:
        """Initialize Vertex AI SDK with project settings."""
        ...
    
    def upload_dataset(
        self,
        config: VertexAIConfig,
        train_path: str,
        val_path: str,
        test_path: str,
        destination_folder: str = "data"
    ) -> Tuple[str, str, str]:
        """Upload datasets to GCS.
        
        Returns:
            Tuple of (train_uri, val_uri, test_uri)
        """
        ...
    
    def create_training_job(
        self,
        config: VertexAIConfig,
        job_config: TrainingJobConfig,
        train_uri: str,
        val_uri: str,
        test_uri: str
    ):
        """Create a training job (not started).
        
        Returns:
            aiplatform.CustomJob instance
        """
        ...
    
    def run_training_job(self, job, sync: bool = True):
        """Run a training job.
        
        Args:
            job: CustomJob instance
            sync: If True, wait for completion
            
        Returns:
            CustomJob instance (completed if sync=True)
        """
        ...
    
    def create_tuning_job(
        self,
        config: VertexAIConfig,
        tuning_config: HyperparameterTuningConfig,
        train_uri: str,
        val_uri: str,
        test_uri: str
    ):
        """Create a hyperparameter tuning job (not started).
        
        Returns:
            aiplatform.HyperparameterTuningJob instance
        """
        ...
    
    def run_tuning_job(self, job, sync: bool = False):
        """Run a tuning job.
        
        Args:
            job: HyperparameterTuningJob instance
            sync: If True, wait for completion (not recommended, takes hours)
            
        Returns:
            HyperparameterTuningJob instance
        """
        ...
    
    def get_best_parameters(self, job) -> Optional[Dict[str, Any]]:
        """Get best hyperparameters from completed tuning job.
        
        Returns:
            Dictionary with trial_id, parameters, and metrics
        """
        ...

print("[SUCCESS] Service protocol defined")

## Summary

This contract layer provides:

1. **Configuration Contracts**:
   - `VertexAIConfig` - Project and credentials settings
   - `TrainingJobConfig` - Training hyperparameters and compute resources
   - `HyperparameterTuningConfig` - Tuning job settings

2. **Model Options**:
   - `ModelType` enum with supported pre-trained models

3. **Service Interface**:
   - `VertexAIService` protocol defining abstract methods

For the **runnable implementation** of this contract, see `vertex_ai.example.ipynb`.